In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk

# load the dataset
dataset = pd.read_csv('./data/train.csv')
dataset.head()

start = 15251
end = 18000

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
stop_words = set(stopwords.words("english"))

def get_category(text: str) -> list:
    res = list()
    soup = BeautifulSoup(text, "html.parser")
    cats = soup.find_all('a', {'href': re.compile('/category/*')})
    for cat in cats:
        res.append(cat.get_text().lower())
    lem = WordNetLemmatizer()
    res = [lem.lemmatize(word) for word in res if not word in  
        stop_words]
    return list(set(res))

res = []
for i in range(0, dataset.shape[0]):
    res.append(get_category(dataset['Page content'][i]))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wmnet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
cats = res

In [4]:
from pytrends.request import TrendReq
from datetime import *
import time
import requests
import random
from termcolor import colored

In [5]:
def get_proxy_list():
    proxies = requests.get("http://list.didsoft.com/get?email=yuwei.shih.tw@gmail.com&pass=73frss&pid=httppremium&showcountry=no&country=SG")
    proxies = proxies.text
    #print(proxies.split("\n")[13:])
    proxies = proxies.split("\n")[:]
    #print(proxies)
    proxy_list = []
    for proxy in proxies:
        proxy_list.append("http://" + proxy)
    proxy_list.remove("http://")
    random.shuffle(proxy_list)
    return proxy_list

# Test
#print(get_proxy_list())
proxy_list = get_proxy_list()
print(int(str(datetime.now()).split(":")[1]) % 30 == 0)
print(proxy_list)

False
['http://157.245.48.174:8080', 'http://150.95.25.110:3128', 'http://45.77.240.139:3128', 'http://178.128.62.211:8080', 'http://139.59.109.156:3128', 'http://139.59.101.223:8080', 'http://119.81.71.27:8123', 'http://159.65.142.133:8080', 'http://66.42.52.89:3128', 'http://119.28.118.116:1080', 'http://139.59.99.119:8080', 'http://167.71.200.182:8118', 'http://157.245.48.206:8080', 'http://134.209.106.109:3128', 'http://206.189.45.191:80', 'http://178.128.100.74:8080', 'http://157.245.56.246:8080', 'http://157.245.153.49:3128', 'http://178.128.106.9:8080', 'http://54.255.188.140:80', 'http://180.210.201.55:3129', 'http://178.128.106.137:8080', 'http://178.128.27.168:8080', 'http://139.99.16.116:3128', 'http://139.59.99.234:3128', 'http://209.97.173.209:8080', 'http://157.245.56.142:8080', 'http://139.180.146.39:80', 'http://209.97.173.184:8080', 'http://157.245.60.143:8080', 'http://139.180.130.167:8080', 'http://178.128.50.159:8080', 'http://178.128.98.43:8080', 'http://159.65.139

In [6]:
def get_date(text: str) -> datetime:
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    date = date.split()
    date = str(date[0] + ' ' + date[1])

    d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return d

def get_trend(word: str, t_start: str, t_end: str) -> list:
    search = word +" "+ t_start +" "+ t_end
    print(colored(search, "yellow"))
    g_trends = TrendReq(tz=360, timeout=(10,25), proxies=get_proxy_list(), retries=4, backoff_factor=0.1)
    g_trends.build_payload([word], timeframe = t_start+' '+t_end, geo='', gprop='')
    trend = g_trends.interest_over_time()
    if trend.size > 0:
        trend.columns = ['0', '1']
        #trend = trend.columns.get_loc()
        # trend.head()
        return trend['0'].values
    else: 
        print(word+" request fail.")
        return []

Proxy error. Changing IP
[ 75  83  67 100  85  95  53  80  81  70  73  85  77  60  82  62  58  80
  73  66  65  74  66  44  54  77  69  56  61  68  63]


In [ ]:
from termcolor import colored

#start = 15251
def trend_num(word: str, pub_time: datetime):
    y_start = str(pub_time).split(' ')[0]
    y_end = str(pub_time + timedelta(days=14)).split(' ')[0]
    global proxy_list
    for i in range(0, 10):
        try:
            y_trend = get_trend(word, y_start, y_end)
            break
        except Exception as e:
            print(e)
            print("Retry step: " + str(i))
            proxy_list = get_proxy_list()
            
    if len(y_trend) > 0:
        return np.sum(y_trend)
    else:
        return 0
    
def trend_rate(word: str, pub_time: datetime):
    now = float(trend_num(word, pub_time))
    #time.sleep(1)
    past = float(trend_num(word, pub_time + timedelta(days=-15)))
    #time.sleep(1)
    if past > 0:
        return now/past
    else:
        return 2
    
def export(check_point):
    g_trend = pd.DataFrame(res)
    g_trend.columns = ["trend_rate"]
    ID = [i for i in range(start, check_point)]
    g_trend['Id'] = ID
    g_trend = g_trend.set_index('Id')
    g_trend.to_csv("trend_%s_%s.csv" % (str(start), str(check_point)))
    
res = []
for i in range(start, end):
    try:
        try:
            date = get_date(dataset["Page content"][i])
            tmp = [0]
            for cat in cats[i]:
                print(colored("Search: " + cat, "red"))
                tmp.append(trend_rate(cat, date))
            res.append(max(tmp))
        except:
            res.append(1)
        print(colored(str(i)+" done", "green"))
        if i%5 == 0:
            export(i+1)
            #time.sleep(30)
        if i == end-1:
            export(end)
    except Exception as e:
        print(e)
        export(i)
        print("Break at: " + str(i))

Search: business
Search: magazine
Search: ipad
Search: medium
15251 done
Search: apple
Search: startup
Search: ebay
Search: ipads
Search: the launchpad
Search: business
Search: ipad
Search: y cominbator
y cominbator request fail.
y cominbator request fail.
Search: uber
Search: iphone
15252 done
Search: google
Search: advertising
Search: amazon
Search: business
Proxy error. Changing IP
Search: facebook
Search: marketing
Search: twitter
15253 done
Search: chart of the day
Search: child mortality
Search: social good
Search: social good chart of the week
social good chart of the week request fail.
social good chart of the week request fail.
Search: millennium development goals
Search: world
15254 done
Search: usc
Search: journalism
Search: gadget
Search: college course
Search: tech
Search: apps and software
Search: google glass
Search: augmented reality
15255 done
Search: privacy
Search: gadget
Search: security
Search: online security
Search: tech
Search: kickstarter
Search: apps and softw

Search: apps and software
15272 done
Search: linux
Search: tech
Search: mobile
Search: apps and software
Search: ubuntu
15273 done
Search: shirtfront
Search: world
Search: tony abbott
Search: australia
15274 done
Search: google
Search: apple
Search: firefox
Search: firefox os
Search: dev & design
dev & design request fail.
dev & design request fail.
Search: encryption
Search: free speech
Search: mobile
Search: social media
Search: smartphone
Search: edward snowden
Search: nsa
Search: kickstarter
Search: top comments
Search: toy story
Search: smartphones
Search: digital rights management
15275 done
Search: u.s.
Search: economics
Search: world
15276 done
Search: twitter
Search: sport
Search: nba
Search: entertainment
15277 done
Search: mobile
Search: nielsen media research
Search: business
Search: research in motion
Search: blackberry 10
Search: blackberry
Search: blackberry messenger
15278 done
Search: google
Search: u.s.
Search: nexus 7
Search: politics
Search: healthcare.gov
Search: i

guest curators,  request fail.
guest curators,  request fail.
Search: social media
Search: mashable photo challenge guest series
mashable photo challenge guest series request fail.
mashable photo challenge guest series request fail.
Search: community
Search: photo challenge series
photo challenge series request fail.
photo challenge series request fail.
Search: photography
Search: instagram
Search: mashable photo challenge
mashable photo challenge request fail.
mashable photo challenge request fail.
15297 done
Search: fitness trackers
Search: mobile
Search: gadget
Search: tech
15298 done
Search: robot
Search: android
Search: lego
Search: gadget
Search: review
Search: usb
Proxy error. Changing IP
Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001ED161357F0>, 'Connection to 159.89.201.15 timed out. (connect timeout=5)'))
Retr

15317 done
Search: google
Search: motorola
Search: nest
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?tz=360&hl=en-US&req=%7B%22category%22%3A+0%2C+%22comparisonItem%22%3A+%5B%7B%22geo%22%3A+%22%22%2C+%22keyword%22%3A+%22nest%22%2C+%22time%22%3A+%222014-01-15+2014-01-29%22%7D%5D%2C+%22property%22%3A+%22%22%7D (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Retry step: 0
Proxy error. Changing IP
Search: mobile
Search: android
Search: gadget
Search: lenovo
Search: waze
Search: opinion
Search: moto x
Search: tech
Proxy error. Changing IP
Proxy error. Changing IP
Search: youtube
Search: apps and software
Search: patent
Search: razr
15318 done
Search: tech
Search: dev and design
Search: imgur
Search: dev & design
dev & design request fail.
15319 done
Search: business
Search: dell
15320 done
Search: video
Search: cute kids
Search: viral video
Search: television
Search: game of thrones
Search: watercooler


Search: dick costolo
Search: ipo
Search: do an ipo right
do an ipo right request fail.
do an ipo right request fail.
15335 done
Search: matchbook
Search: amazon
Search: mobile
Search: e-books
Search: tech
Search: marketing
Search: mashreads
mashreads request fail.
Search: kindle
15336 done
Search: tech
Search: nokia
Search: nokia lumia 625
Search: mobile
Search: lumia 625
15337 done
Search: surface tablet
Search: mobile
Search: gadget
Search: surface mini
Search: tech
Search: microsoft
Search: satya nadella
15338 done
Search: bbc america
Search: entertainment
Search: television
Search: orphan black
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?tz=360&hl=en-US&req=%7B%22category%22%3A+0%2C+%22comparisonItem%22%3A+%5B%7B%22geo%22%3A+%22%22%2C+%22keyword%22%3A+%22orphan+black%22%2C+%22time%22%3A+%222014-06-22+2014-07-06%22%7D%5D%2C+%22property%22%3A+%22%22%7D (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out'

...

Search: u.s.
15847 done
Search: u.s.
Search: entertainment
Search: movie
Search: unbroken
Search: louis zamperini
Search: angelina jolie
Proxy error. Changing IP
Proxy error. Changing IP
15848 done
Search: merger
Search: viacom
Search: medium
Search: video
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001ED17965E80>, 'Connection to 178.128.106.137 timed out. (connect timeout=5)'))
Retry step: 0
Proxy error. Changing IP
Proxy error. Changing IP
Search: movie
Search: harry potter
Search: television
Search: mashable must reads
mashable must reads request fail.
mashable must reads request fail.
Search: comcast
Search: business
Search: time warner
Search: cbs corp.
cbs corp. request fail.
Search: rupert murdoch
Search: directv
15849 done
Search: business
Proxy error. Changing IP
Proxy error. Changing IP
The request failed: Google returned a response wi

viral newsletter sent request fail.
viral newsletter sent request fail.
Search: serial
Search: saturday night live
Search: christmas
15865 done
Search: business
Search: brand
Search: marketing
Search: mcdonalds
15866 done
Search: invention
Proxy error. Changing IP
The request failed: Google returned a response with code 429.
Retry step: 0
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001ED160D36A0>, 'Connection to 157.245.192.16 timed out. (connect timeout=5)'))
Retry step: 1
Proxy error. Changing IP
Proxy error. Changing IP
Proxy error. Changing IP
Search: watercooler
The request failed: Google returned a response with code 429.
Retry step: 0
Proxy error. Changing IP
Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedH

Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:719)'),))
Retry step: 2
Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001ED17C33438>, 'Connection to 178.128.106.137 timed out. (connect timeout=5)'))
Retry step: 3
Proxy error. Changing IP
Search: tara reid
The request failed: Google returned a response with code 429.
Retry step: 0
Search: sharknado
15877 done
Search: movie
Search: conversation
Search: disney
Search: hashtags
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/widgetdata/multiline?req=%7B%22requestOptions%22%3A+%7B%22category%22%3A+0%2C+%22backend%22%3A+%22IZG%22%2C+%22property%22%3A+%22%22

Search: tornado
Search: world
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?tz=360&hl=en-US&req=%7B%22category%22%3A+0%2C+%22comparisonItem%22%3A+%5B%7B%22geo%22%3A+%22%22%2C+%22keyword%22%3A+%22world%22%2C+%22time%22%3A+%222013-05-21+2013-06-04%22%7D%5D%2C+%22property%22%3A+%22%22%7D (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Retry step: 0
The request failed: Google returned a response with code 429.
Retry step: 1
Proxy error. Changing IP
Proxy error. Changing IP
Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by SSLError(SSLError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:719)'),))
Retry step: 2
Proxy error. Changing IP
Proxy error. Changing IP
15900 done
Search: u.s.
Search: politics
Search: television
Search: jfk50
Search: jfk
Proxy error. Changing IP
Search: world
The request faile

HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:719)'),))
Retry step: 1
Proxy error. Changing IP
Search: march madness
Search: sport
Search: world
Search: college sports
Search: legal
Search: union
15914 done
Search: u.s.
Search: lego
Search: drone
Search: gadget
Search: tech
Search: blimp
Search: diy
15915 done
Search: nvidia
Search: gaming
Search: android
Search: gadget
Proxy error. Changing IP
Search: shield
Search: tech
Search: twitch
Search: project shield
15916 done
Search: lifestyle
HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /?geo=US (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001ED17BCB2B0>, 'Connection to 157.245.56.142 timed out. (connect timeout=5)'))
Retry step: 0
Proxy error. Changing IP
Proxy error. Changing IP
HTTPSConnectionPool(host='trends.google.com

In [ ]:
#res[1] = 2 0 o
n = 0
print(np.mean(res))
for i in range(start, start+len(res)):
    if (res[i] > float(np.mean(res))) and (1 == dataset["Popularity"][i]):
        n = n+1
    elif (res[i] < float(np.mean(res))) and (-1 == dataset["Popularity"][i]):
        n = n+1
    print(str(i) + ": \t" + str(dataset["Popularity"][i]) +"\t"+str(res[i]))
print("accu: " + str(n/len(res)))

In [ ]:
# def export():
#     g_trend = pd.DataFrame(res)
#     g_trend.columns = ["trend_rate"]
#     ID = [i for i in range(start, end)]
#     g_trend['Id'] = ID
#     g_trend = g_trend.set_index('Id')
#     g_trend.to_csv("trend.csv")

In [ ]:
"trend_%s_%s.csv" % (str(start), str(55))